# Joins

In [2]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%reload_ext sql
# Verbinde Dich zu einer in - Memory Datenbank
%sql sqlite:///
%sql PRAGMA foreign_keys = ON

 * sqlite:///
Done.


[]

# Die verschiedenen Joins

Es gibt verschiedene Arten Joins zu erklären. Ich bevorzuge immer noch folgendes Bild:

![](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=SQL_Joins.png)



In [3]:
%%sql

DROP TABLE IF EXISTS eins;
DROP TABLE IF EXISTS zwei;

CREATE TABLE eins(id INT PRIMARY KEY, val CHAR(1));
INSERT INTO eins VALUES(1,'A'), (2,'B'), (3,'C');
CREATE TABLE zwei(id INT PRIMARY KEY, val2 CHAR(1));
INSERT INTO zwei VALUES(2,'D'), (3,'E'), (4,'F');

 * sqlite:///
Done.
Done.
Done.
3 rows affected.
Done.
3 rows affected.


[]

# Aufgabe 1
Führen sie an den Tabellen `eins, zwei` verschiedene `JOINS` durch, um ein Gefühl für die veränderte Ergebnissmenge zu erhalten. Nachdem beide Tabellen gleich benannte `id` Attribute haben können Sie bei einem `JOIN` auf diese einen `NATURAL JOIN` oder `JOIN using(id)` oder `JOIN ON(eins.id = zwei.id)` verwenden.


#### Aufgabe 1.1 Implementieren Sie einen `INNER` Join in den drei oben genannten Varianten

In [ ]:
%%sql

SELECT * FROM eins NATURAL JOIN zwei;

 * sqlite:///
Done.


id,val,val2
2,B,D
3,C,E


In [ ]:
%%sql

SELECT * FROM eins JOIN zwei USING (id);

 * sqlite:///
Done.


id,val,val2
2,B,D
3,C,E


In [ ]:
%%sql

SELECT * FROM eins JOIN zwei on(eins.id = zwei.id);

 * sqlite:///
Done.


id,val,id_1,val2
2,B,2,D
3,C,3,E


#### Aufgabe 1.2 Implementieren Sie einen `LEFT Join`

In [9]:
%%sql

SELECT * FROM eins LEFT JOIN zwei USING(id);

 * sqlite:///
Done.


id,val,val2
1,A,None
2,B,D
3,C,E


#### Aufgabe 1.3 Implementieren Sie einen `RIGHT Join`

In [10]:
%%sql

SELECT * FROM eins RIGHT JOIN zwei USING(id);

 * sqlite:///
Done.


id,val,val2
2,B,D
3,C,E
4,None,F


#### Aufgabe 1.4 Implementieren Sie `A - B`

In [12]:
%%sql

SELECT * FROM eins LEFT JOIN zwei USING(id) WHERE zwei.id IS NULL;

 * sqlite:///
Done.


id,val,val2
1,A,None


#### Aufgabe 1.5 Implementieren Sie `B - A`

In [14]:
%%sql

SELECT * FROM zwei LEFT JOIN eins using(id) WHERE eins.id IS NULL;

 * sqlite:///
Done.


id,val2,val
4,F,None


#### Aufgabe 1.5 Implementieren Sie einen `FULL OUTER JOIN`

In [18]:
%%sql

SELECT id, val, val2 FROM eins LEFT JOIN zwei USING(id) UNION ALL SELECT id, val, val2 FROM zwei LEFT JOIN eins USING(id)


 * sqlite:///
Done.


id,val,val2
1,A,None
2,B,D
3,C,E
2,B,D
3,C,E
4,None,F


In [17]:
%%sql

SELECT DISTINCT * FROM (SELECT id, val, val2 FROM eins LEFT JOIN zwei USING(id) UNION ALL SELECT id, val, val2 FROM zwei LEFT JOIN eins USING(id)) as gemeinsam ORDER BY id;

 * sqlite:///
Done.


id,val,val2
1,A,None
2,B,D
3,C,E
4,None,F


# Aufgabe 2

Die Beispiele aus Aufgabe 1 waren bewusst abstrakt gewählt. Was heißt die jeweilige Form von Joins denn auf einen konkreten Use-Case? Was würde jede einzelne Join form bedeuten, wenn es um Stundenten und Bücher ginge, die Studenten ausleihen können?

- Inner Join: 
- Left Outer Join:
- Right Outer Join:
- Student - Buch:
- Buch - Student:
- FULL OUTER JOIN:

# Aufgabe 3

Betrachten wir folgene Modellierung:

![](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=bib.png)

In [19]:
%%sql

DROP TABLE IF EXISTS leiht_aus;
DROP TABLE IF EXISTS ist_ueberfaellig;
DROP TABLE IF EXISTS student;
DROP TABLE IF EXISTS buch;

CREATE TABLE student(
    matrikelnummer INT PRIMARY KEY NOT NULL,
    name TEXT NOT NULL
);

CREATE TABLE buch(
    isbn TEXT PRIMARY KEY NOT NULL,
    name TEXT NOT NULL
);

CREATE TABLE leiht_aus(
    matrikelnummer INT NOT NULL,
    isbn TEXT NOT NULL,
    von DATE NOT NULL,
    bis DATE,
    PRIMARY KEY(matrikelnummer, isbn, von),
    FOREIGN KEY(matrikelnummer) REFERENCES student(matrikelnummer),
    FOREIGN KEY(isbn) REFERENCES buch(isbn)
);

CREATE TABLE ist_ueberfaellig(
    matrikelnummer INT NOT NULL,
    isbn TEXT NOT NULL,
    gebuer INT,
    PRIMARY KEY(matrikelnummer, isbn),
    FOREIGN KEY(matrikelnummer) REFERENCES student(matrikelnummer),
    FOREIGN KEY(isbn) REFERENCES buch(isbn)
);

 * sqlite:///
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [20]:
%%sql
INSERT INTO student VALUES(1,'Elser'), (2, 'Maier'), (3, 'Huber');
INSERT INTO buch VALUES(1,'Datenbanken'), (2, 'Ulysses'), (3, 'Das Niebelungenlied');
INSERT INTO ist_ueberfaellig VALUES(1,2,10), (2,2,15), (2,3,22);
INSERT INTO leiht_aus VALUES(1,1,'01.10.2021',NULL), (1,2,'01.11.2021',NULL), (2,3,'01.11.2021','11.11.2021');

 * sqlite:///
3 rows affected.
3 rows affected.
3 rows affected.
3 rows affected.


[]

In [28]:
%%sql

SELECT * FROM student;

 * sqlite:///
Done.


matrikelnummer,name
1,Elser
2,Maier
3,Huber


In [29]:
%%sql

SELECT * FROM buch;

 * sqlite:///
Done.


isbn,name
1,Datenbanken
2,Ulysses
3,Das Niebelungenlied


In [30]:
%%sql

SELECT * FROM ist_ueberfaellig;

 * sqlite:///
Done.


matrikelnummer,isbn,gebuer
1,2,10
2,2,15
2,3,22


In [31]:
%%sql

SELECT * FROM leiht_aus;

 * sqlite:///
Done.


matrikelnummer,isbn,von,bis
1,1,01.10.2021,None
1,2,01.11.2021,None
2,3,01.11.2021,11.11.2021


#### Aufgabe 3.1

Entwickeln Sie einen JOIN, der angibt welche Studenten welche Bücher ausgeliehen haben und welche Gebüren da noch zu zahlen sind.

In [35]:
%%sql

-- Student name, Book name, (von, bis), Fee

SELECT student.name AS 'Student Name', buch.name AS 'Book Name', ist_ueberfaellig.gebuer AS 'Fee'
FROM student
LEFT JOIN leiht_aus USING(matrikelnummer)
LEFT JOIN buch USING(isbn)
LEFT JOIN ist_ueberfaellig ON(student.matrikelnummer = ist_ueberfaellig.matrikelnummer AND buch.isbn = ist_ueberfaellig.isbn)

 * sqlite:///
Done.


Student Name,Book Name,Fee
Elser,Datenbanken,None
Elser,Ulysses,10
Maier,Das Niebelungenlied,22
Huber,None,None


#### Aufgabe 3.2

Geben Sie nur ausleihen aus, die > 20 Euro Gebüren haben

In [36]:
%%sql

-- Student name, Book name, (von, bis), Fee

SELECT student.name AS 'Student Name', buch.name AS 'Book Name', ist_ueberfaellig.gebuer AS 'Fee'
FROM student
LEFT JOIN leiht_aus USING(matrikelnummer)
LEFT JOIN buch USING(isbn)
LEFT JOIN ist_ueberfaellig ON(student.matrikelnummer = ist_ueberfaellig.matrikelnummer AND buch.isbn = ist_ueberfaellig.isbn)
WHERE ist_ueberfaellig.gebuer > 20

 * sqlite:///
Done.


Student Name,Book Name,Fee
Maier,Das Niebelungenlied,22
